In [1]:
import pandas as pd
import numpy as np

theme_version_signal = pd.read_csv("theme_version_signal.csv")
theme_persistence = pd.read_csv("theme_persistence.csv")


In [2]:
latest_version = theme_version_signal["RC_ver"].max()

latest_snapshot = theme_version_signal[
    theme_version_signal["RC_ver"] == latest_version
].copy()


In [3]:
latest_snapshot = latest_snapshot.merge(
    theme_persistence[["theme", "Is_Persistent"]],
    on="theme",
    how="left"
)


In [4]:
latest_snapshot["Reach"] = (
    0.6 * latest_snapshot["Normalized_Signal"] +
    0.4 * (latest_snapshot["Review_Count"] /
           latest_snapshot["Review_Count"].max())
)


In [5]:
latest_snapshot["Impact"] = (
    (5 - latest_snapshot["Avg_Rating"]) / 4
)

latest_snapshot.loc[
    latest_snapshot["Is_Regression"], "Impact"
] *= 1.2

latest_snapshot.loc[
    latest_snapshot["Is_Persistent"], "Impact"
] *= 1.2


In [6]:
latest_snapshot["Confidence"] = (
    0.7 * (latest_snapshot["Review_Count"] /
           latest_snapshot["Review_Count"].max()) +
    0.3 * latest_snapshot["Is_Persistent"].astype(int)
)


In [7]:
effort_map = {
    "Playback Reliability": 4,
    "Navigation & Home Feed": 3,
    "Library & Playlist Control": 3,
    "Free vs Premium Friction": 2,
    "UI & Content Surfaces": 2,
    "Performance & Media Issues": 4
}

latest_snapshot["Effort"] = latest_snapshot["theme"].map(effort_map)


In [8]:
latest_snapshot["Priority_Score"] = (
    latest_snapshot["Reach"] *
    latest_snapshot["Impact"] *
    latest_snapshot["Confidence"]
) / latest_snapshot["Effort"]


In [9]:
priority_backlog = (
    latest_snapshot
    .sort_values("Priority_Score", ascending=False)
    .loc[:, [
        "theme",
        "Priority_Score",
        "Reach",
        "Impact",
        "Confidence",
        "Effort",
        "Is_Regression",
        "Is_Persistent"
    ]]
)

priority_backlog


,theme,Priority_Score,Reach,Impact,Confidence,Effort,Is_Regression,Is_Persistent
5,UI & Content Surfaces,0.203941,0.496226,0.826957,0.993966,2,False,True
1,Free vs Premium Friction,0.202716,0.494156,0.827948,0.990948,2,False,True
3,Navigation & Home Feed,0.138343,0.500710,0.828879,1.000000,3,False,True
2,Library & Playlist Control,0.137026,0.498410,0.827273,0.996983,3,False,True
4,Performance & Media Issues,0.103386,0.500480,0.826293,1.000000,4,False,True
0,Playback Reliability,0.101970,0.496226,0.826957,0.993966,4,False,True


In [10]:
priority_backlog.to_csv("priority_backlog.csv", index=False)
